In [1]:
import os
!pip3 install glob
import glob
import dill
import pathlib
import pandas as pd
import numpy as np
!pip3 install pydub
import pydub
!pip3 install python_speech_features
import python_speech_features
from utils import * 
import re

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
base_folder = pathlib.Path("/content/drive/My Drive/validated_clips/validated_clip/")
out_folder =  pathlib.Path("/content/drive/My Drive/validated_clips/preprocessed/")
outfiles = os.listdir(out_folder)
already_done = set()

for file in outfiles:
  thing = re.compile('[a-z]')
  already_done.add(file[:thing.search(file).start()])

folders = os.listdir(base_folder)

In [29]:
todo = ["2.000000"]
for folder in folders:
  #if folder in already_done:
  #  print(f'skipping {folder}')
  # continue
  if folder not in todo:
    print(f'skipping{folder}')
    continue

  print(folder)
  inner_folder = base_folder / folder
  globby = str(inner_folder)+"/*.mp3"
  downloaded_files = glob.glob(globby)
  #print(downloaded_files)

  filepath_dict = {pathlib.Path(filepath).name : filepath for filepath in downloaded_files}
  print(filepath_dict)

  dev = pd.read_csv("/content/drive/My Drive/tsvs/dev.tsv", sep="\t")
  train = pd.read_csv("/content/drive/My Drive/tsvs/train.tsv", sep="\t")
  test = pd.read_csv("/content/drive/My Drive/tsvs/test.tsv", sep="\t")
  splits = {
      "dev": dev,
      "train": train, 
      "test": test
  }

  #only files in current folder
  for key in splits:
    print(key)
    print(splits[key].shape)
    splits[key]["path"] = splits[key]["path"].map(filepath_dict)
    splits[key] = splits[key][splits[key]['path'].notna()]
    print(splits[key].shape)

  #only ones with accents
  for key in splits:
    splits[key].dropna(axis=0, subset=["accent"], inplace=True)
  

  downloaded_audios_dict = {key : [pydub.AudioSegment.from_mp3(f) for f in splits[key]['path']] for key in splits}
  max_lengths = [np.max([len(sample) for sample in samples]) for samples in downloaded_audios_dict.values() if samples != []]
  if max_lengths == []:
    print("I cry")
    continue
  max_audio_length = np.max(max_lengths)
  padded_audio_dict = {key: [zero_pad_in_end(audio, max_audio_length) for audio in audios] for key, audios in downloaded_audios_dict.items()}

  labels_dict = {key: splits[key]['accent'].to_numpy() for key in splits}
  paths_dict ={key: [f for f in splits[key]['path']] for key in splits}
  mfccs_padded_dict = {key: np.array([extract_mfcc(audio) for audio in padded_audios]) for key, padded_audios in padded_audio_dict.items()}

  label_filename = folder+"label.pkl"
  paths_filename = folder+"paths.pkl"
  mfccs_padded_filename = folder+"mfccs_padded.pkl"

  label_filepath = out_folder / label_filename
  paths_filepath = out_folder / paths_filename
  mfccs_padded_filepath = out_folder / mfccs_padded_filename

  with open(label_filepath , "wb") as f:
    dill.dump(labels_dict, f)
  with open(paths_filepath , "wb") as f:
    dill.dump(paths_dict, f)
  with open(mfccs_padded_filepath , "wb") as f:
    dill.dump(mfccs_padded_dict, f)

skipping103.000000
skipping101.000000
skipping100.000000
skipping105.000000
skipping104.000000
skipping106.000000
skipping1.000000
skipping107.000000
skipping102.000000
skipping10.000000
skipping114.000000
skipping116.000000
skipping11.000000
skipping109.000000
skipping115.000000
skipping110.000000
skipping111.000000
skipping112.000000
skipping113.000000
skipping108.000000
skipping120.000000
skipping122.000000
skipping117.000000
skipping124.000000
skipping119.000000
skipping125.000000
skipping121.000000
skipping12.000000
skipping123.000000
skipping118.000000
skipping127.000000
skipping130.000000
skipping129.000000
skipping134.000000
skipping128.000000
skipping13.000000
skipping133.000000
skipping131.000000
skipping126.000000
skipping132.000000
skipping142.000000
skipping138.000000
skipping141.000000
skipping14.000000
skipping135.000000
skipping143.000000
skipping140.000000
skipping137.000000
skipping136.000000
skipping139.000000
skipping145.000000
skipping151.000000
skipping148.000000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


skipping206.000000
skipping200.000000
skipping202.000000
skipping199.000000
skipping20.000000
skipping205.000000
skipping22.000000
skipping27.000000
skipping26.000000
skipping21.000000
skipping25.000000
skipping207.000000
skipping24.000000
skipping23.000000
skipping208.000000
skipping28.000000
skipping35.000000
skipping34.000000
skipping37.000000
skipping33.000000
skipping36.000000
skipping3.000000
skipping29.000000
skipping32.000000
skipping30.000000
skipping31.000000
skipping44.000000
skipping43.000000
skipping39.000000
skipping38.000000
skipping46.000000
skipping4.000000
skipping42.000000
skipping40.000000
skipping45.000000
skipping41.000000
skipping5.000000
skipping51.000000
skipping52.000000
skipping48.000000
skipping47.000000
skipping55.000000
skipping50.000000
skipping53.000000
skipping49.000000
skipping54.000000
skipping60.000000
skipping57.000000
skipping61.000000
skipping6.000000
skipping58.000000
skipping64.000000
skipping62.000000
skipping63.000000
skipping59.000000
skippin

In [0]:
print(len(already_done))

208


In [0]:
print(len(folders))

208


In [0]:
preprocessed_folder = pathlib.Path('/content/drive/My Drive/validated_clips/preprocessed/')
mfccs_padded = {"dev":[], "train":[], "test":[]}
for i in range(1,209):
  number = '{:.6f}'.format(i)
  mfcc_filename = number+"mfccs_padded.pkl"
  mfcc_filepath = preprocessed_folder / mfcc_filename

  with open(mfcc_filepath, "rb") as f:
    mfcc_local = dill.load(f)
  mfccs_padded['dev'].extend(mfcc_local['dev'])
  mfccs_padded['train'].extend(mfcc_local['train'])
  mfccs_padded['test'].extend(mfcc_local['test'])

In [4]:
max_lengths = [np.max([len(sample) for sample in samples]) for samples in mfccs_padded.values() if samples != []]
print(max_lengths)
max_length = np.max(max_lengths)

[5579, 6234, 6234]


In [5]:
print(max_length)

6234


In [0]:
for i in range(1,209):
  number = '{:.6f}'.format(i)
  mfcc_filename = number+"mfccs_padded.pkl"
  mfcc_filepath = preprocessed_folder / mfcc_filename

  with open(mfcc_filepath, "rb") as f:
    mfcc_local = dill.load(f)
  local_padded = {key: [np.pad(audio, [(0, max_length-audio.shape[0]),(0,0)], mode="constant", constant_values=0) for audio in audios] for key, audios in mfcc_local.items()}
  out_filename = number+"mfccs_padded_real.pkl"
  out_filepath = preprocessed_folder / out_filename
  with open(out_filepath , "wb") as f:
    dill.dump(local_padded, f)

In [0]:
overall_padded= {key: np.stack( overall_padded[key], axis=0 ) for key in overall_padded.keys()}

In [1]:
print(stackyboi)

NameError: ignored

In [0]:

overall_filepath = preprocessed_folder / "overall_mfcc_padded.pkl"
with open(mfcc_filepath, "wb") as f:
    mfcc_local = dill.write(overall_padded, f)